In [2]:
import csv
import os
from datetime import datetime
import json
import psycopg2
import sys
from ipyleaflet import Map, WKTLayer
import re

sys.path.insert(0,'..')

from scripts.database import get_conn_pool, execute, fetch_all
from scripts.routing import route_nogo, route_standard

In [3]:
# connect to postgis database
pool = get_conn_pool({
    'host': 'localhost',
    'port': 5432,
    'user': 'postgres',
    'password': 'postgres',
    'database': 'xlab-routing'
})

In [4]:
# set start and end points

# START COORDINATES
start = [36.73858,-1.16696]
# start_lat = -1.271346
# start_long = 36.751293
start_lat = start[1]
start_long = start[0]

# END COORDINATES
end = [37.01403,-1.28636]
# end_lat = -1.190509
# end_long = 36.726833
end_lat = end[1]
end_long = end[0]

In [16]:
# set nogo layers
nogo_layer_list = ['hgis_admin3', 'hgis_bridges_tunnels']

# set nogo layer queries (needs to be in same order as nogo_layer_list)
nogo_layer_query_list = ["hgis_admin3.adm3_name = 'Kitisuru'", 
                         "hgis_bridges_tunnels.objectid = 810 OR hgis_bridges_tunnels.objectid = 811"]

# set nogo point layer buffer distance (default is .0005 in degrees or 50 meters) 
nogo_point_buffer = .0005

# set nogo polyline layer buffer distance (default is .0001 in degrees or 10 meters)
nogo_line_buffer = .0001

# set nogo cost percentage weight (default is 1000000000)
nogo_cost = 1000000000

# set noise for randomness (default is 0)
noise = .75


In [19]:
# get route that avoids nogo geometry
route = route_nogo(nogo_layer_list, start_long, start_lat, end_long, end_lat, nogo_layer_query_list, 
                   nogo_point_buffer, nogo_line_buffer, nogo_cost, noise)
wkt = route[0]
length = route[1]

NOGO QUERY:  SELECT ST_GEOMFROMEWKT(ST_COLLECT(a.shape)) FROM (SELECT ST_FORCE2D(shape) as shape FROM hgis_admin3 WHERE hgis_admin3.adm3_name = 'Kitisuru' UNION SELECT ST_FORCE2D(ST_BUFFER(shape, 0.0005)) as shape FROM hgis_bridges_tunnels WHERE hgis_bridges_tunnels.objectid = 810 OR hgis_bridges_tunnels.objectid = 811) as a
START NODE:  7889
END NODE:  21825


In [20]:
# Display route

m = Map(center=[start_lat, end_long], zoom=10)
wlayer = WKTLayer(
    wkt_string=wkt,
    hover_style={"fillColor": "red"},
)
m.add_layer(wlayer)

m

Map(center=[-1.16696, 37.01403], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…